<a href="https://colab.research.google.com/github/MarcoMosconi/RDF-Knowledge-Graph/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# requirements
!pip install -q rdflib sparqlwrapper owlrl gdown pandas wikipedia-api

In [ ]:
import gdown
import os
import pandas as pd
from rdflib import Graph, URIRef, BNode, Literal, FOAF, RDF, RDFS, XSD, Namespace
from SPARQLWrapper import SPARQLWrapper
import math
from dateutil import parser
import wikipediaapi
from urllib.parse import urlparse
from time import sleep
import owlrl
sparql = SPARQLWrapper("https://dbpedia.org/sparql")

In [ ]:
!pip freeze > requirements.txt

In [ ]:
# create directory for data
if not os.path.exists("./data/"):
    os.mkdir("./data/")

In [ ]:
# Download data
gdown.download(id="1UILSMenvqzFMsIhV6l4ZV1opJpRQZnwC", output="./data/rym_top_5000_all_time.csv", quiet=True)
gdown.download(id="1UNJ2iDX-xoWIekSA0ZwoqNOk3eAZjqrp", output="./data/albumlist.csv", quiet=True)

'./data/albumlist.csv'

# **KRR Module 2 2022/23 Project**

The project consists in the creation of an RDF knowledge graph starting from tabular data (two CSV files loaded below).

You should cover all of the following steps.

### **`1. Create an RDFLib Graph from data contained in CSV files`**

In steps:
1) Load the CSV files in two pandas DataFrames and join them
2) Create an RDFLib `Graph` and add triples from the data of the dataframes

Useful links:

- [Pandas user guide](https://pandas.pydata.org/docs/user_guide/index.html) and [documentation](https://pandas.pydata.org/docs/reference/), e.g.:
    - [10 minutes to pandas](https://pandas.pydata.org/docs/user_guide/10min.html)
    - [Join dataframes](https://pandas.pydata.org/docs/user_guide/10min.html#join)
    - [Splitting and replacing strings](https://pandas.pydata.org/docs/user_guide/text.html#splitting-and-replacing-strings)
    - [Apply a function over an axis of a DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html?highlight=apply#pandas.DataFrame.apply)

In [ ]:
df1 = pd.read_csv('data/albumlist.csv', encoding='mac_roman')
df1.head()

,Number,Year,Album,Artist,Genre,Subgenre
0,1,1967,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,Rock,"Rock & Roll, Psychedelic Rock"
1,2,1966,Pet Sounds,The Beach Boys,Rock,"Pop Rock, Psychedelic Rock"
2,3,1966,Revolver,The Beatles,Rock,"Psychedelic Rock, Pop Rock"
3,4,1965,Highway 61 Revisited,Bob Dylan,Rock,"Folk Rock, Blues Rock"
4,5,1965,Rubber Soul,The Beatles,"Rock, Pop",Pop Rock


In [ ]:
df2 = pd.read_csv('data/rym_top_5000_all_time.csv')
df2.head()

,Ranking,Album,Artist Name,Release Date,Genres,Descriptors,Average Rating,Number of Ratings,Number of Reviews
0,1.0,OK Computer,Radiohead,16 June 1997,"Alternative Rock, Art Rock","melancholic, anxious, futuristic, alienation, ...",4.23,"70,382",1531
1,2.0,Wish You Were Here,Pink Floyd,12 September 1975,"Progressive Rock, Art Rock","melancholic, atmospheric, progressive, male vo...",4.29,"48,662",983
2,3.0,In the Court of the Crimson King,King Crimson,10 October 1969,"Progressive Rock, Art Rock","fantasy, epic, progressive, philosophical, com...",4.30,"44,943",870
3,4.0,Kid A,Radiohead,3 October 2000,"Art Rock, Experimental Rock, Electronic","cold, melancholic, futuristic, atmospheric, an...",4.21,"58,590",734
4,5.0,To Pimp a Butterfly,Kendrick Lamar,15 March 2015,"Conscious Hip Hop, West Coast Hip Hop, Jazz Rap","political, conscious, poetic, protest, concept...",4.27,"44,206",379


In [ ]:
df1.shape, df2.shape

((500, 6), (5000, 9))

In [ ]:
df = pd.merge(df1, df2, on= 'Album', how='outer')
all_genre = []
for index, row in df.iterrows():
  if type(row['Genre']) == str:
    genre = row['Genre'].split(', ') #this is necessary for the cases where there are more genres for one album (like Rock, Pop for Rubber Soul)
    for g in genre:
      all_genre.append(g)

for index, row in df.iterrows():
  if type(row['Genres']) == str:
    genres = row['Genres'].split(', ')
    for g2 in genres:
      if g2 in all_genre:
          if pd.isna(row['Genre']):
            df.loc[index,'Genre'] = g2
          elif g2 not in row['Genre']:
            df.loc[index, 'Genre'] += ', '+g2
      else:
        if pd.isna(row['Subgenre']):
          df.loc[index,'Subgenre'] = g2
        elif g2 not in row['Subgenre']:
          df.loc[index, 'Subgenre'] += ', '+g2

df['Artists'] = df['Artist'].combine_first(df['Artist Name'])
df = df.drop(columns = ['Ranking', 'Number', 'Year', 'Genres','Artist','Artist Name'])
df.head()


,Album,Genre,Subgenre,Release Date,Descriptors,Average Rating,Number of Ratings,Number of Reviews,Artists
0,Sgt. Pepper's Lonely Hearts Club Band,Rock,"Rock & Roll, Psychedelic Rock, Psychedelic Pop...",1 June 1967,"psychedelic, playful, melodic, male vocals, wa...",4.13,"43,576",863.0,The Beatles
1,Pet Sounds,Rock,"Pop Rock, Psychedelic Rock, Baroque Pop",16 May 1966,"Wall of Sound, warm, bittersweet, love, romant...",4.18,"36,305",727.0,The Beach Boys
2,Revolver,Rock,"Psychedelic Rock, Pop Rock, Psychedelic Pop",5 August 1966,"psychedelic, melodic, male vocals, drugs, ecle...",4.23,"43,178",1160.0,The Beatles
3,Highway 61 Revisited,Rock,"Folk Rock, Blues Rock, Singer/Songwriter",30 August 1965,"poetic, cryptic, surreal, sarcastic, male voca...",4.17,"26,338",604.0,Bob Dylan
4,Rubber Soul,"Rock, Pop",Pop Rock,3 December 1965,"melodic, love, male vocals, bittersweet, roman...",4.03,"33,508",721.0,The Beatles


In order to handle the Genre-Subgenre-Genres I decided to maintain Genre and Subgenre of the first dataframe and divide Genres of the second into those two categories.

I create a list where I put all the elements of 'Genre' of the first dataframe, then I iterate the elements of Genres and I check if the element is present in the list of Genre.

If the current Genres value is present in the list I consider it as a Genre value, so if there is not a Genre value for the album, this Genres values become the Genre one, while if there are already one or more values, and the Genres one is different, I add it (if it is already present I just go on).

If it is not present in the list I consider it as a Subgenre value, so if there is not a Subgenre value for the album, this Genres values become the Subgenre one, while if there are already one or more values, and the Genres one is different, I add it (if it is already present I just go on).

Since the Artist category from the first dataframe and the Artist name from the second one are equivalent, I merge them into one Artists category
Since Ranking and Number make sense only in their dataframes, I can eliminate them in the unified dataframe, while I decide to drop Year from the first dataframe and maintain Release Date from the second one, since it contains more or less the same information and it is related to much more albums.
Since I divided Genres into Genre and Subgenre, and I created the new category Artists from Artist and Artist Name, I dropped those categories.


Create an RDFLib `Graph` and add triples from the data of the dataframes**

For example, add triples like the following (but not only):

- `[album] dbp:artist [artist]`,
- `[album] rdfs:label [album_name]`,
- `[album] rdf:type [genre]`

Make sure you use URIs and Literals correctly.

In [ ]:
#I create the graph and define the namespace, binding them
graph = Graph()
DBP = Namespace('https://dbpedia.org/property/')
DBR = Namespace('http://dbpedia.org/resource/')
DBO = Namespace("http://dbpedia.org/ontology/")
ex = Namespace("http://example.org/")
graph.bind("", ex)
graph.bind("dbp", DBP)
graph.bind('dbo', DBO)
graph.bind('dbr', DBR)

In [ ]:
#artist
#I iterate through the dataframe, keeping as subject the Album and if the corresponding artist was present I added the triple
#with it as object and DBP.artist as predicate (I use replace() so that the values could fit inside the dbpedia URIs)
for index, row in df.iterrows():
  subject = DBR[row['Album'].replace(' ', '_').replace('"', '').replace('&','and')]
  if type(row['Artists']) == str: #there can be NaN values which must not be considered
    graph.add((subject, DBP.artist, DBR[row['Artists'].replace(' ', '_').replace('"','')]))

In [ ]:
#genre
for index, row in df.iterrows():
  subject = DBR[row['Album'].replace(' ', '_').replace('"', '').replace('&','and')]  #I always maintain as subject of the triple the Album items
  string = ''
  if type(row['Genre']) == str:
    for letter in row['Genre'].replace(' / ', ', ').replace('&', 'and').replace(', and', ','): #I iterate all the letters in the Genre row so that I can replace the necessary characters (I divided Funk / Soul into two genres, in many cases the dbpedia URIs work with 'and' but not with &, and the last replace is for World, and Country)
      string += letter          #add the letters to the string and I split it when there is a ,
    obj = string.split(", ")
    new_obj = []  #create a list where I put the genre(s)
    for element in obj:
      element = element.replace(' ', '_') #this is needed so that cases like Hip Hop become Hip_Hop
      new_obj.append(element)
    for genre in new_obj:
      graph.add((subject, DBP.genre, DBR[genre]))

In [ ]:
#label
#I do the same procedure as artist, but this time considering the object as a string, an XSD value
for index, row in df.iterrows():
  subject = DBR[row['Album'].replace(' ', '_').replace('"', '').replace('&','and')]
  graph.add((subject, RDFS.label, Literal(row['Album'], datatype=XSD.string)))

In [ ]:
#subgenre
#I do the same procedure as genre
for index, row in df.iterrows():
  subject = DBR[row['Album'].replace(' ', '_').replace('"', '').replace('&','and')]
  string = ''
  if type(row['Subgenre']) == str:
    for letter in row['Subgenre'].replace(' / ', ', ').replace('&', 'and').replace(', and', ','):
      string += letter
    obj = string.split(", ")
    new_obj = []
    for element in obj:
      element = element.replace(' ', '_')
      new_obj.append(element)
    for subgenre in new_obj:
      graph.add((subject, DBP.subgenre, DBR[subgenre]))

In [ ]:
#release date
#I do the same procedure as label, but in this case it is a date and not a string
for index, row in df.iterrows():
  subject = DBR[row['Album'].replace(' ', '_').replace('"', '').replace('&','and')]
  if type(row['Release Date']) == str:
    release_date = parser.parse(row['Release Date']).date()
    graph.add((subject, ex.release_date, Literal(release_date, datatype=XSD.date)))

In [ ]:
#average rating
#Same procedure, this time an integer
for index, row in df.iterrows():
    subject = DBR[row['Album'].replace(' ', '_').replace('"', '').replace('&','and')]
    if not math.isnan(row['Average Rating']): #Since they are integer, I can avoid having Nan values with math.isnan()
      graph.add((subject, ex.avg_rating, Literal(row['Average Rating'], datatype=XSD.integer)))

In [ ]:
#number of ratings
#Same procedure of the others
for index, row in df.iterrows():
  subject = DBR[row['Album'].replace(' ', '_').replace('"', '').replace('&','and')]
  if type(row['Number of Ratings']) == str: #Since there is the , I have to use type == string
    num_rating = row['Number of Ratings'].replace(',','')
    graph.add((subject, ex.num_rating, Literal(num_rating, datatype=XSD.integer)))

In [ ]:
#number of reviews
#Same procedure as before
for index, row in df.iterrows():
  subject = DBR[row['Album'].replace(' ', '_').replace('"', '').replace('&','and')]
  if not math.isnan(row['Number of Reviews']):
    graph.add((subject, ex.num_reviews, Literal(int(row['Number of Reviews']), datatype=XSD.integer)))

In [ ]:
#descriptors
#Same procedure as genre/subgenre, this time with the object as a string
for index, row in df.iterrows():
  subject = DBR[row['Album'].replace(' ', '_').replace('"', '').replace('&','and')]
  string = ''
  if type(row['Descriptors']) == str:
    for letter in row['Descriptors']:
      string += letter
    obj = string.split(", ")
    new_obj = []
    for element in obj:
      element = element.replace(' ', '_')
      new_obj.append(element)
    for descriptor in new_obj:
      graph.add((subject, ex.descriptors, Literal(descriptor, datatype=XSD.string)))

In [ ]:
graph.serialize(destination="data/1stGraph.ttl", format='turtle')

<Graph identifier=Nedd8011fc77447d1b5af5ac5c5f9112b (<class 'rdflib.graph.Graph'>)>

### **`2. Integrate your data with DBpedia's`**

You MUST extend the knowledge you already have on your local KG with the following from DBpedia:
- band members
- city of birth of members
- country of birth of members
- founding city of the band
- founding country of the band

You can also add more information regarding albums (optional)



Recommendations and hints:
- How to find IRIs from DBpedia that match the entities in the local KG?
  1) guessing the IRI with simple tricks:
     - the album 'Rubber Soul' on DBpedia is `dbr:Rubber_Soul`, and this 'translation' may work for other albums (don't do it manually)
     - the album 'Led Zeppelin' on DBpedia is `dbr:Led_Zeppelin_(album)` as the same name is also associated to the band
  2) through their `rdfs:label`: for instance, the artist [Amy Winehouse](https://en.wikipedia.org/wiki/Amy_Winehouse) has 'Amy Winehouse' as label on DBpedia and therfore after the matching the IRI of the entity can be extracted
  3) using the [DBpedia lookup endpoint](https://github.com/dbpedia/lookup). The following request tries to find the DBpedia entity for the artist [Fabrizio De Andrè](https://en.wikipedia.org/wiki/Fabrizio_De_Andr%C3%A9): https://lookup.dbpedia.org/api/search/KeywordSearch?QueryString=fabrizio%20de%20andre&format=json&MaxHits=5
- restrict the search to entities of type 'Album', 'Band', 'Country', etc, in order to avoid overloading DBPedia's endpoint (see [here](http://mappings.dbpedia.org/server/ontology/classes/) for a list of DBPedia classes) and thus avoid high latency or being banned
- note that sometimes useful properties to reach entities have the `dbp` prefix which is different from `dbo` ([more info](https://parklize.blogspot.com/2016/05/dbpedia-difference-between-dbo-and-dbp.html)). It's ok to use both
- here is a **non-exaustive** list of useful properties: `dbp:hometown`, `dbo:birthPlace`, `dbo:country`, `dbo:formerBandMember`
- how to find properties that link two entities of our interest?
  1) by checking the [DBpedia ontology](https://akswnc7.informatik.uni-leipzig.de/dstreitmatter/archivo/dbpedia.org/ontology/2023.05.12-020000/ontology_type=parsed.owl) manually
  2) by checking the DBpedia ontology with Protégé
  3) using the DBpedia resource pages and exploiting your knowledge (or Wikipedia information). For instance, let us consider [Metallica](https://en.wikipedia.org/wiki/Metallica), I know that [Dave Mustaine](https://en.wikipedia.org/wiki/Dave_Mustaine) was a member, therefore I ask myself: How are these two entities linked in the DBpedia KG? On the [DBpedia page of Metallica](https://dbpedia.org/page/Metallica) we can find that the property used is `dbo:formerBandMember` thus I can use this property to get band members
  4) running SPARQL queries like the following in the DBpedia endpoint:
      ```
      SELECT DISTINCT ?p
      WHERE {
        ?s a dbo:Band .
        ?o a dbo:MusicalArtist .
        ?s ?p ?o .
      }
      ```
  5) if you are curious enough you can check [ABSTAT](http://abstat.disco.unimib.it/about.html), a tool for KG exploration with a nice interface
- please notice that DBpedia is not perfect so is reccomendable to check the type of the candidates before including them

In [ ]:
graph = Graph().parse("./data/1stGraph.ttl")


In [ ]:
artists = df['Artists'].dropna().unique()
for artist in artists:
  art = DBR[artist.replace(' ', '_').replace('"', '').replace('&','and')]
  cond = f"{{ <{art}> a dbo:Band; dbo:bandMember ?b . optional {{<{art}> dbo:hometown ?h}} optional {{<{art}> dbo:hometown/dbo:country ?o}} optional {{?b dbo:birthPlace ?p}} optional {{?b dbo:birthPlace/dbo:country ?c}}}}"
  cond1 = f"{{ <{art}> a dbo:Band; dbo:formerBandMember ?b . optional {{<{art}> dbo:hometown ?h}} optional {{<{art}> dbo:hometown/dbo:country ?o}} optional {{?b dbo:birthPlace ?p}} optional {{?b dbo:birthPlace/dbo:country ?c}}}}"
  q = f"construct  {{ <{art}> a dbo:Band ; dbo:bandMember ?b ; dbp:hometown ?h ; dbo:origin ?o. ?b dbo:birthPlace ?p; dbo:country ?c}}\nwhere {{ {cond} union\n {cond1} }}"
  sparql.setQuery(q)
  sparql.setReturnFormat("xml")
  results = sparql.query().convert()
  if not results:
    art1 = art + "_(band)"
    cond = f"{{ <{art1}> a dbo:Band; dbo:bandMember ?b . optional {{<{art1}> dbo:hometown ?h}} optional {{<{art1}> dbo:hometown/dbo:country ?o}} optional {{?b dbo:birthPlace ?p}} optional {{?b dbo:birthPlace/dbo:country ?c}}}}"
    cond1 = f"{{ <{art1}> a dbo:Band; dbo:formerBandMember ?b . optional {{<{art1}> dbo:hometown ?h}} optional {{<{art1}> dbo:hometown/dbo:country ?o}} optional {{?b dbo:birthPlace ?p}} optional {{?b dbo:birthPlace/dbo:country ?c}}}}"
    q = f"construct  {{ <{art}> a dbo:Band ; dbo:bandMember ?b ; dbp:hometown ?h ; dbo:origin ?o. ?b dbo:birthPlace ?p; dbo:country ?c}}\nwhere {{ {cond} union\n {cond1} }}"
    sparql.setQuery(q)
    sparql.setReturnFormat("xml")
    results = sparql.query().convert()
  graph += results


I drop the Nan values and to make the query faster I consider the artists just once with unique()

For the query retrieving all the required information I define two similar condition and put them together with union. The difference between the two is that in cond1 there is dbo:bandMember and in cond2 dbo:formerBandMember,
(which in the construct are all shown as dbo:bandMember). this is necessary to deal with all cases, like The Velvet Underground which have only former band member, The Rolling Stones which have both former and current band member or other cases where the artist is defined as a dbo:Band but has not band members.

I use some property paths, for Origin (it is better in cases like The Rolling Stones where origin has as value 'London, England' while with the property path I get only the country), and for the country where the band members where born.

Since there are artists like the Queen for which the corresponding dbpedia URI brings to the disambiguate page and returns nothing, I build another identical query, with the only difference that I add to the artist _(band), so that it is addressed correctly even for cases like that one.




In [ ]:
graph.serialize(destination="data/queryGraph.ttl")

<Graph identifier=Nc46ed922b1d447beb65aebab444093fb (<class 'rdflib.graph.Graph'>)>

### **`3. Gathering Information from unstructured data`**

Knowledge can be gathered from unstructured data as well as structured data. A great encyclopaedic source is Wikipedia whose articles follow a very strict pattern. For instance, the first lines about a person are reserved for biographical data and his or her occupation, therefore we can exploit it to understand the role of a band member and add this information to our KG.

In steps:
1) get the article from Wikipedia and extract the summary. You can use [Wikipedia-api](https://pypi.org/project/Wikipedia-API/) (see the code below)
2) extract the band member role (guitarist, bassist, keyboardist, singer, drummer, etc) using rules
3) add this information to the KG

Recommendations and hints:
- some artists have multiple roles, add them all
- some artists have multiple bands, it is not required to understand the role for each band, keep it simple.
- extraction rules can be very simple (simple string matchers + if-else)  or more complex ones (e.g., regex, ChatGPT requests), it doesn't matter in terms of points.


In [ ]:
graph = Graph().parse("./data/queryGraph.ttl")

In [ ]:

wiki_wiki = wikipediaapi.Wikipedia('en')
page_py = wiki_wiki.page('Kurt Cobain')

if page_py.exists():
    summary = page_py.summary[:200]
    print("Page - Summary: %s" % summary)
else:
    print("Page does not exist.")

Page - Summary: Kurt Donald Cobain (February 20, 1967 – c. April 5, 1994) was an American musician who was the co-founder, lead vocalist, guitarist and primary songwriter of the rock band Nirvana. Through his angst-f


In [ ]:
roles = ['singer', 'founder', 'co-founder', 'vocalist', 'leadvocalist', 'co-leadvocalist', 'songwriter','guitarist', 'bassist', 'drummer', 'keyboardist', 'pianist', 'saxophonist', 'trumpeter', 'trombonist', 'violinist', 'percussionist', 'lyricist', 'composer', 'producer']


In [ ]:
for s, p, o in graph.triples((None, DBO.bandMember, None)):
  sleep(1)
  parsed_bM = urlparse(o)
  bandMember = parsed_bM.path.split('/')[-1].replace('_',' ')
  if len(bandMember) > 0:
    page_py = wiki_wiki.page(bandMember)
    if page_py.exists():
      summary = page_py.summary[:200]
      summary = summary.replace(',','').replace('(','').replace(')','').replace('ad vo', 'advo')
      for string in roles:
        if string in summary.split():
          string = string.replace('advo', 'ad vo')
          graph.add((o, ex.role, Literal(string, datatype=XSD.string)))

After having created the array with all the possible roles, I iterate through the triples with dbo:bandMember as predicate, considering the object (the band member). The object is a URI, so I divide it into its components, take the path and split it replacing the _ with a space in order to have the original string. There was just one case of an empty string, so I put len > 0.
I iterate through the list of roles (in which I written leadvocalist and co-leadvocalist so that, unifying those letters also in the summary, if someone is a vocalist it doesn't identify him also a lead vocalist or a co-lead vocalist), and if there are roles matching with words in the summary it adds them to the graph.

I need to add a time.sleep(1) because I have throttling issues.


In [ ]:
graph.serialize(destination="data/wikiGraph.ttl")

<Graph identifier=Ncbae3b92b2fe433db8f1b2cc73535983 (<class 'rdflib.graph.Graph'>)>

### **`4. Include a small ontology and a  genres taxonomy`**

Using RDFlib, add classes such as Person, MusicAlbum, and so on to your KG. Add the domain and range for each property you use.

This will be needed later for inferring the types of the entities of your KG.

In [ ]:
graph = Graph().parse("./data/wikiGraph.ttl")

In [ ]:
triples = [
    (ex.Person, RDF.type, RDFS.Class),
    (ex.BandMember, RDF.type, RDFS.Class),
    (ex.MusicAlbum, RDF.type, RDFS.Class),
    (ex.Genre, RDF.type, RDFS.Class),
    (ex.Subgenre, RDF.type, RDFS.Class),
    (ex.MusicalArtist, RDF.type, RDFS.Class),
    (ex.Singer, RDF.type, RDFS.Class),
    (ex.BandMember, RDFS.subClassOf, ex.Person),

    (DBP.artist, RDF.type, RDF.Property),
    (DBP.artist, RDFS.domain, ex.MusicAlbum),
    (DBP.artist, RDFS.range, ex.MusicalArtist),

    (DBP.genre, RDF.type, RDF.Property),
    (DBP.genre, RDFS.domain, ex.MusicAlbum),
    (DBP.genre, RDFS.range, ex.Genre),

    (DBP.subgenre, RDF.type, RDF.Property),
    (DBP.subgenre, RDFS.domain, ex.MusicAlbum),
    (DBP.subgenre, RDFS.range, ex.Genre),

    (RDFS.label, RDF.type, RDF.Property),
    (RDFS.label, RDFS.domain, ex.MusicAlbum),
    (RDFS.label, RDFS.range, XSD.string),

    (ex.release_date, RDF.type, RDF.Property),
    (ex.release_date, RDFS.domain, ex.MusicAlbum),
    (ex.release_date, RDFS.range, XSD.date),

    (ex.avg_rating, RDF.type, RDF.Property),
    (ex.avg_rating, RDFS.domain, ex.MusicAlbum),
    (ex.avg_rating, RDFS.range, XSD.integer),

    (ex.num_rating, RDF.type, RDF.Property),
    (ex.num_rating, RDFS.domain, ex.MusicAlbum),
    (ex.num_rating, RDFS.range, XSD.integer),

    (ex.num_reviews, RDF.type, RDF.Property),
    (ex.num_reviews, RDFS.domain, ex.MusicAlbum),
    (ex.num_reviews, RDFS.range, XSD.integer),

    (ex.descriptors, RDF.type, RDF.Property),
    (ex.descriptors, RDFS.domain, ex.MusicAlbum),
    (ex.descriptors, RDFS.range, XSD.string),
]
for triple in triples:
  graph.add(triple)

Moreover, the first dataframe has 'Genre' and 'Subgenre' columns.
Extract a genres taxonomy from the dataset and include it in your RDF KG. This will be needed later for inferring genres from subgenres.

E.g.: add triples to the graph like:

- `[subgenre] rdfs:subClassOf [genre]`.
- `[album] rdf:type [genre]`.

This should be done for all genres and subgenres (with an iteration, not manually).

In [ ]:
all_genre = list(set(all_genre))

all_subgenre = []
for index, row in df.iterrows():
  if type(row['Subgenre']) == str:
    subgenre = row['Subgenre'].split(', ')
    for subg in subgenre:
      all_subgenre.append(subg)

all_subgenre = list(set(all_subgenre))

for gen in all_genre:
  for subgen in all_subgenre:
    if gen in subgen:
      gen = gen.replace(' ','_')
      subgen = subgen.replace(' ','_')
      graph.add((DBR[subgen], RDFS.subClassOf, DBR[gen]))

for s, p, o in graph.triples((None, DBP.genre, None)):
  graph.add((s, RDF.type, o))
for s, p, o in graph.triples((None, DBP.subgenre, None)):
  graph.add((s, RDF.type, o))

I consider the all_genre list I created at the beginning, making it a set, so that I get the unique values, and then again a list.
I also create an equal list for the subgenres.

I iterate through the two lists in such a way that if an element in the list genre is present in an element of the list subgenre (e.g., Rock and Psychedelic Rock), then the element of the subgenre list is added to the graph as subClassof the element of the genre list.

In order to add to the graph that the album are rdf:type the genres, since at the beginning I added them with the predicate dbp:genre/dbp:subgenre, I just iterate through those triple and duplicate them changing the predicate.


In [ ]:
graph.serialize(destination='data/ontGraph.ttl')

<Graph identifier=N05ca24b50cff46d9ba500e903ea8da4b (<class 'rdflib.graph.Graph'>)>

### **`5. Materialize inferences`**

Add to your KG inferences that you can make from the data on your KG (e.g., with SPARQL queries, or with OWLRL, ...).

**<ins>Some</ins>**  **<ins>examples</ins>**
- if you have that `[album] rdf:type [genre1]` and `[genre1] rdfs:subClassOf [genre2]`, add the triple `[album] rdf:type [genre2]`. Same for domains, ranges, subclasses, subproperties, OWL features, etc.
- if the nationality of a group member is missing, you can infer it from the nationality of the band

Run a few SPARQL queries on the graph to see the inferred triples.


In [ ]:
graph = Graph().parse("./data/ontGraph.ttl")

In [ ]:
g = """insert {?member a :BandMember} \nwhere {?x ns1:bandMember ?member}"""
graph.update(g)
owlrl.DeductiveClosure(owlrl.OWLRL_Semantics).expand(graph)
q = """insert {?member ns1:country ?c} \nwhere {?band ns1:bandMember ?member; ns1:origin ?c . filter not exists {select ?member where {?member ns1:country ?cc}}}"""
graph.update(q)
h = """insert {?bandMember :band ?band} \nwhere {?band ns1:bandMember ?bandMember}"""
graph.update(h)
a = """insert {?artist a :Singer} \nwhere {?artist a :MusicalArtist . filter not exists {?artist a ns1:Band}}"""
graph.update(a)


I add inferences with the reasoner and four SPARQL queries.

The important inference I get from the reasoner is that if an album is of rdf:type genre1, and genre1 is subclassof genre2, then the album is of type genre2.

With the first query I add the class BandMember to all the band members, and the reasoner will then add to them also the class Person, since when I added the ontology I wrote that BandMember is subclassof Person.

With the second query I added the country of a band member, if missing, from the nationality of his band.

With the third query I add to the band members the corresponding band.

With the fourth query I add the class Singer to the MusicalArtists which are not Bands.





In [ ]:
graph.serialize(destination = 'data/infGraph.ttl')

<Graph identifier=Nb83cb223c929406c958e19cb1190c888 (<class 'rdflib.graph.Graph'>)>

In [ ]:
check_inf1 = graph.query("select ?bandMember where {?bandMember a :BandMember} limit 300")
for bandMember in check_inf1:
  bM = bandMember['bandMember'].split('/')[-1].replace('_', ' ')
  print(f"{bM}")

%22Sneaky%22 Pete Kleinow
Martin Rev
Les Smith
Tony Bellotto
Larry Wallis
Mikael Stanne
Parker Kindred
Alex Rosamilia
Tracey Thorn
Carl Verheyen
Moses Archuleta
Kazik Staszewski
Mike Kinsella
Stic.man
Mike Mangini
Hamish Glencross
Manuel Göttsching
Necrobutcher
Dave Sharp
Kenney Jones
John Avila
Allen Ravenstine
David Hidalgo
Jan Visser
Jeremy Enigk
Lars Frederiksen
Paul James (Canadian musician)
Phil Lynott
Chris Caffery
Mark Olson (musician)
Donna Dresch
James Stelfox
Tuomas Peippo
Derek Shulman
Will Champion
Ahrue Luster
Phil Miller
Jeff Lords
Raymond Pettibon
Mark Clarke (musician)
Frédéric Leclercq
Bruno Ruder
Ed O'Brien
Wayne Shorter
Mark Matejka
Niclas Engelin
Denis D'Amour
Lori Lewis
Nick Bassett (musician)
Daddy G
Dave Mattacks
Mike Scalzi
Ebbot Lundberg
Carl-Michael Eide
Vincent Cavanagh
Casey Grillo
Rat Skates
Russell Allen
George Kollias (drummer)
Marcus Siepen
Vincent Crane
Christofer Johnsson
Masta Ace
Tom Skinner (drummer)
Travis Stever
Jean-Jacques Kravetz
Raekwon
Bryan

In [ ]:
check_inf2 = graph.query("select ?artist where {?artist a :Singer} limit 300")
for artist in check_inf2:
  art = artist['artist'].split('/')[-1].replace('_', ' ')
  print(f"{art}")

 Pierre-Laurent Aimard
John Carpenter & Alan Howarth
John Zorn
Fabrizio De André
Richard Jacques
Charly García
Jay Munly
Ijahman
Scanner
SOPHIE
Crack
Changó
光田康典 [Yasunori Mitsuda]
Deströyer 666
Fela
Hugh Mundell
Monks
 Carmine Coppola
Kevin Ayers
Dusk
The Mamas and the Papas
e.s.t.
芸能山城組 [Geinoh Yamashirogumi]
Księżyc
Richard Dawson
Kenny Dorham
フィッシュマンズ
The Bill Dixon Orchestra
Demigod
Dwight Yoakam
Sonny Sharrock
Envy
Lil Wayne
Colin Newman
Max Roach and His Chorus and Orchestra
Егор и Опизденевшие [Yegor i Opizdenevshie]
Gorement
Frank Sinatra
Einstuerzende Neubauten
Kashiwa Daisuke
 Herbert von Karajan
Earth Wind & Fire
 Elisabeth Schwarzkopf
Chuck Person
Exuma
Weyes Blood
 植松伸夫 [Nobuo Uematsu]
Canned Heat
Sly & The Family Stone
Truth and Janey
The Max Roach Trio featuring the Legendary Hassan
Nuno Canavarro
Whitney Houston
Magdalena Bay
Lô Borges
Shivkumar Sharma, Brij Bhushan Kabra & Hariprasad Chaurasia
Milton Nascimento & Lô Borges
 Maria Cristina Kiehr
KISS
Ibrahim Ferrer
Har

### **`6. Query the Graph`**

Query your graph in a way that:
- the usefulness of having added information from dbpedia is highlighted
- inference is exploited


**<ins>Some</ins>**  **<ins>examples</ins>** with the intentions:


| ID | Query | Intention/insight |
|----|------|-------------------|
| 1  | which bands have at least one member with a different nationality? | just curiosity |
| 2  | Which are the artist with many bands? | to spot session musicians, long-career artists, 'eclectic' artists |
| 3  | which are the countries with more [ Latin Rock ]( https://en.wikipedia.org/wiki/Latin_rock ) bands? and what about [ Black Metal ]( https://en.wikipedia.org/wiki/Black_metal )? | I suppose that latin rock is more popular in Latin America countries while Black Metal in Scandinavian countries, but I may be wrong! |
| 4  | Is there any artist who is a member of both a [ Britpop ]( https://en.wikipedia.org/wiki/Britpop ) band and a [ Hip hop ]( https://en.wikipedia.org/wiki/Hip_hop_music ) band | maybe to spot bands/artists with a strong musical contamination? e.g., [Damon Albarn ](https://en.wikipedia.org/wiki/Damon_Albarn) |
| 5  | Which is the city with more bands in absolute? and what if we consider only [ Grunge ]( https://en.wikipedia.org/wiki/Grunge )? | maybe to discover a music scene or just a very influential city. |
| 6  | Give me some band/artist from the same city of [ 2Pac ]( https://en.wikipedia.org/wiki/Tupac_Shakur ) | could be a way to find similar musical artists. What about adding also a time period and the musical genre? |
| 7  | which are the bands with Argentine and Spanish members? | maybe because I love both the Argentine and Spanish sound and I hope there is something that matches both? e.g., [Los Rodriguez](https://en.wikipedia.org/wiki/Los_Rodr%C3%ADguez) |




**Feel free to play and get cool insights**

In [ ]:
graph = Graph().parse("./data/infGraph.ttl")

In [ ]:
print("WHICH BANDS HAVE AT LEAST ONE MEMBER WITH A DIFFERENT NATIONALITY?")
answer_1 = graph.query("select ?band ?mem1 ?mem2 ?c1 ?c2 where {?band ns1:bandMember ?mem1,?mem2. ?mem1 ns1:country ?c1. ?mem2 ns1:country ?c2 . filter (?c1 != ?c2 && str(?mem1) > str(?mem2))}")
for answer in answer_1:
  band = answer['band'].split('/')[-1].replace('_', ' ')
  mem1 = answer['mem1'].split('/')[-1].replace('_', ' ')
  mem2 = answer['mem2'].split('/')[-1].replace('_', ' ')
  c1 = answer['c1'].split('/')[-1].replace('_', ' ')
  c2 = answer['c2'].split('/')[-1].replace('_', ' ')
  print(f"The band {band} has band member {mem1} born in {c1} and band member {mem2} born in {c2}")

WHICH BANDS HAVE AT LEAST ONE MEMBER WITH A DIFFERENT NATIONALITY?
The band Annihilator has band member Randy Black born in Germany and band member Big John Bates born in Canada
The band Annihilator has band member Randy Black born in Germany and band member Dave Padden born in Canada
The band Annihilator has band member Randy Black born in Germany and band member Jeff Waters born in Canada
The band Annihilator has band member Randy Black born in Germany and band member Joe Comeau born in United States
The band Annihilator has band member Randy Black born in Germany and band member Mike Mangini born in United States
The band Madvillain has band member Madlib born in United States and band member MF Doom born in United Kingdom
The band Sepultura has band member Wagner Lamounier born in Brazil and band member Derrick Green born in United States
The band Tangerine Dream has band member Michael Hoenig born in Germany and band member Edgar Froese born in Russia
The band Tangerine Dream has 

In [ ]:
print("WHICH COUNTRIES HAVE MORE PSYCHEDELIC ROCK BANDS?")
answer_2 = graph.query("select ?country (count(?band) as ?numBand) where {?band ns1:origin ?country; a dbr:Psychedelic_Rock} group by ?country order by desc(?numBand)")
for answer in answer_2:
  country = answer['country'].split('/')[-1].replace('_', ' ')
  num_band = answer['numBand']
  print(f"{country} has {num_band} Psychedelic Rock bands")

WHICH COUNTRIES HAVE MORE PSYCHEDELIC ROCK BANDS?
United States has 3 Psychedelic Rock bands
United Kingdom has 2 Psychedelic Rock bands
Germany has 1 Psychedelic Rock bands
Brazil has 1 Psychedelic Rock bands


In [ ]:
print("WHICH ARE BANDS WITH BRAZILIAN AND AMERICAN MEMBERS?")
answer_3 = graph.query("select ?band ?mem1 ?mem2 where {?band ns1:bandMember ?mem1, ?mem2. ?mem1 ns1:country dbr:Brazil. ?mem2 ns1:country dbr:United_States}")
for answer in answer_3:
  band = answer['band'].split('/')[-1].replace('_', ' ')
  mem1 = answer['mem1'].split('/')[-1].replace('_', ' ')
  mem2 = answer['mem2'].split('/')[-1].replace('_', ' ')
  print(f"{band} has {mem1} from Brazil and {mem2} from the United States")


WHICH ARE BANDS WITH BRAZILIAN AND AMERICAN MEMBERS?
Weather Report has Airto Moreira from Brazil and Greg Errico from the United States
Weather Report has Airto Moreira from Brazil and Peter Erskine from the United States
Weather Report has Airto Moreira from Brazil and Alphonse Mouzon from the United States
Weather Report has Airto Moreira from Brazil and Alphonso Johnson from the United States
Weather Report has Airto Moreira from Brazil and Chester Thompson from the United States
Weather Report has Airto Moreira from Brazil and Don Alias from the United States
Weather Report has Airto Moreira from Brazil and Jaco Pastorius from the United States
Weather Report has Airto Moreira from Brazil and Narada Michael Walden from the United States
Weather Report has Airto Moreira from Brazil and Wayne Shorter from the United States
Sepultura has Andreas Kisser from Brazil and Derrick Green from the United States
Pat Metheny Group has Armando Marçal from Brazil and Cuong Vu from the United St

In [ ]:
print("WHICH ARE THE BANDS WITH THE MOST MEMBERS (FORMER MEMBERS INCLUDED)?")
answer_4 = graph.query("select ?band (count(?bandMember) as ?numMem) where {?band ns1:bandMember ?bandMember} group by ?band order by desc(?numMem)")
for answer in answer_4:
  band = answer['band'].split('/')[-1].replace('_', ' ')
  num_mem = answer['numMem']
  print(f"{band} has {num_mem} members")


WHICH ARE THE BANDS WITH THE MOST MEMBERS (FORMER MEMBERS INCLUDED)?
The Jazz Messengers has 68 members
Funkadelic has 32 members
Santana has 28 members
Procol Harum has 26 members
Broken Social Scene has 24 members
Chicago has 24 members
The Temptations has 22 members
Squeeze has 22 members
Weather Report has 22 members
The Lounge Lizards has 21 members
Thinking Plague has 20 members
New York Dolls has 19 members
The Mars Volta has 19 members
The Tony Williams Lifetime has 19 members
Fairport Convention has 19 members
Yes has 19 members
Buena Vista Social Club has 19 members
Tangerine Dream has 18 members
Ministry has 18 members
Fleetwood Mac has 18 members
Misfits has 17 members
Steeleye Span has 17 members
The Allman Brothers Band has 17 members
Black Flag has 17 members
Steve Miller Band has 17 members
The Blues Brothers has 17 members
The Roots has 17 members
Iron Maiden has 17 members
Silver Jews has 16 members
Camel has 16 members
Swans has 16 members
Pat Metheny Group has 16 me

In [ ]:
print("WHICH BANDS HAVE OR HAVE HAD A PIANIST?")
answer_5 = graph.query("select ?band ?bandMember where {?band ns1:bandMember ?bandMember. ?bandMember :role \"pianist\"^^<http://www.w3.org/2001/XMLSchema#string>}")
for answer in answer_5:
  band = answer['band'].split('/')[-1].replace('_', ' ')
  bandMember = answer['bandMember'].split('/')[-1].replace('_', ' ')
  print(f"{band} have {bandMember} as pianist")

WHICH BANDS HAVE OR HAVE HAD A PIANIST?
The Hellacopters have Anders Lindström as pianist
The Jazz Messengers have Benny Green (pianist) as pianist
Little Feat have Bill Payne as pianist
The Jazz Messengers have Bobby Timmons as pianist
Coldplay have Chris Martin as pianist
Squeeze have Christopher Holland as pianist
Los Jaivas have Claudio Parra as pianist
The Jazz Messengers have Donald Brown (musician) as pianist
Antony and the Johnsons have Doveman as pianist
The Alan Parsons Project have Eric Woolfson as pianist
Esbjörn Svensson Trio have Esbjörn Svensson as pianist
Magma have Francis Moze as pianist
Procol Harum have Gary Brooker as pianist
The Jazz Messengers have George Cables as pianist
The Jazz Messengers have Horace Silver as pianist
The Decemberists have Jenny Conlee as pianist
Stratovarius have Jens Johansson as pianist
Mahavishnu Orchestra have Jim Beard as pianist
At the Drive-In have Jim Ward (musician) as pianist
The Jazz Messengers have Joanne Brackeen as pianist
The 

In [ ]:
print("WHICH ARE THE ARTISTS BORN IN THE SAME CITY THEIR BAND WAS FOUNDED?")
answer_6 = graph.query("select ?bandMember ?band ?city where {?band ns1:bandMember ?bandMember. ?bandMember ns1:birthPlace ?city. ?band ns2:hometown ?hometown filter (?city = ?hometown) }")
for answer in answer_6:
  artist = answer['bandMember'].split('/')[-1].replace('_', ' ')
  band = answer['band'].split('/')[-1].replace('_', ' ')
  city = answer['city'].split('/')[-1].replace('_', ' ')
  print(f"{artist} was born in {city}, where his band, {band}, was founded")

WHICH ARE THE ARTISTS BORN IN THE SAME CITY THEIR BAND WAS FOUNDED?
Luis Alberto Spinetta was born in Buenos Aires, where his band, Almendra, was founded
Luis Alberto Spinetta was born in Buenos Aires, where his band, Invisible, was founded
Luis Alberto Spinetta was born in Buenos Aires, where his band, Spinetta Jade, was founded
Tadeusz Nalepa was born in Poland, where his band, Breakout, was founded
Jan Borysewicz was born in Wrocław, where his band, Lady Pank, was founded
Neil Cicierega was born in Boston, where his band, Lemon Demon, was founded
Artur Rojek was born in Mysłowice, where his band, Myslovitz, was founded
Artur Rojek was born in Poland, where his band, Myslovitz, was founded
Artur Rojek was born in Mysłowice, where his band, Lenny Valentino, was founded
Madlib was born in Oxnard, California, where his band, Quasimoto, was founded
Wagner Lamounier was born in Belo Horizonte, where his band, Sarcófago, was founded
Wagner Lamounier was born in Belo Horizonte, where his ba

In [ ]:
print("WHO ARE THE FUNK SINGERS?")
answer_7 = graph.query("select ?singer where {?album a dbr:Funk; dbp:artist ?singer. ?singer a :Singer}")
for answer in answer_7:
  singer = answer['singer'].split('/')[-1].replace('_', ' ')
  print(f"{singer} is a funk singer")

WHO ARE THE FUNK SINGERS?
Prince is a funk singer
Phil Spector is a funk singer
Demon Fuzz is a funk singer
Diana Ross & The Supremes is a funk singer
Etta James is a funk singer
Amy Winehouse is a funk singer
Phil Spector is a funk singer
Michael Jackson is a funk singer
Albert King is a funk singer
Gil Scott-Heron & Brian Jackson is a funk singer
Al Green is a funk singer
Barry White is a funk singer
Otis Redding is a funk singer
TLC is a funk singer
Fred Wesley and the J.B.'s is a funk singer
Prince is a funk singer
Dr. John is a funk singer
Dusty Springfield is a funk singer
George Michael is a funk singer
Sly & The Family Stone is a funk singer
Elvis Presley is a funk singer
Smokey Robinson & The Miracles is a funk singer
Paul Simon is a funk singer
Sly & The Family Stone is a funk singer
Elton John is a funk singer
Al Green is a funk singer
Dr. John, the Night Tripper is a funk singer
Marvin Gaye is a funk singer
James Brown is a funk singer
Al Green is a funk singer
Earth, Wind 

### **`Notes`**
- attempts to enrich the data (e.g., using Wikipedia articles, other KGs and data sources) will be positively evaluated, but only if enrichment in steps 2 and 3 has been sufficiently explored
- sometimes matching local entities with DBpedia entities is difficult: do not worry too much about that, we will give more importance to the choices and motivations rather than the end result of the matching
- as you can see, the instructions leave enough freedom, however in case of non-obvious clarifications or technical issues contact me at renzo.alvaprincipe@unimib.it or use the course forum. Questions about how to do the exercise will be ignored
- plagiarism will be **<ins>severely</ins>** punished for both parties